## 네이버 블로그 수집

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import unicodedata
import time
import random
import pandas as pd

In [8]:
driver_path = 'C:/Users/Playdata/Desktop/dev/setup/chromdriver.exe'
driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(3)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_1496\2318196145.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [ ]:
#https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate=2023-05-04&endDate=2023-05-11&keyword=%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91

In [9]:
def get_blog_search_url(keyword, start_date, end_date, page):
    url = 'https://section.blog.naver.com/Search/Post.naver?'
    url += f'pageNo={page}&rangeType=WEEK&orderBy=sim&'
    url += f'startDate={start_date}&endDate={end_date}&keyword={keyword}'

    return url

In [10]:
keyword = '제주도맛집'
start_date = '2023-05-04'
end_date = '2023-05-11'
page = 1
url = get_blog_search_url(keyword, start_date, end_date, page)

In [11]:
# 페이지 이동
driver.get(url)


In [12]:
html = driver.page_source
# len(html)
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# len(soup.select('div > a.desc_inner'))

7

In [13]:
one_page_blog_urls = [a['href'] for a in soup.select('div > a.desc_inner')]
one_page_blog_urls

['https://blog.naver.com/kidart01/223095054870',
 'https://blog.naver.com/the_yoons/223095690267',
 'https://blog.naver.com/skdud2028/223094524030',
 'https://blog.naver.com/namnom8888/223094738155',
 'https://blog.naver.com/wkdu200/223093053779',
 'https://blog.naver.com/aldksaldks93/223093063963',
 'https://blog.naver.com/namokran66/223094838570']

In [14]:
#1. 7개 블로그 URL 추출하는 함수
def get_one_page_blog_urls(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    one_page_blog_urls = [a['href'] for a in soup.select('div > a.desc_inner')]

    return one_page_blog_urls

In [19]:
#2. 1~10페이지로 반복하여 이동하며 70개의 블로그 url 추출
page_num = 10

total_blog_url_list = []
page_nums_list = []

for i in range(1, page_num+1):
    url = get_blog_search_url(keyword, start_date, end_date, i)
    driver.get(url)
    time.sleep(3)

    one_page_blog_urls = get_one_page_blog_urls(driver) # 7개
    total_blog_url_list += one_page_blog_urls #.extend() 사용해도 됌
    page_nums_list += [i] * len(one_page_blog_urls)

In [20]:
len(total_blog_url_list)

70

In [21]:
url_dict = dict()
url_dict['page'] = page_nums_list
url_dict['blog_url'] = total_blog_url_list

In [25]:
df = pd.DataFrame(url_dict)

In [26]:
df.to_excel('네이버블로그_제주도맛집_URL.xlsx', index=False)

- 각 블로그 페이지 데이터 추출(iframe 처리)

In [6]:
df_url = pd.read_excel('네이버블로그_제주도맛집_URL.xlsx')
df_url.head()

,page,blog_url
0,1,https://blog.naver.com/kidart01/223095054870
1,1,https://blog.naver.com/the_yoons/223095690267
2,1,https://blog.naver.com/skdud2028/223094524030
3,1,https://blog.naver.com/namnom8888/223094738155
4,1,https://blog.naver.com/wkdu200/223093053779


In [28]:
driver.get(df_url['blog_url'][0])

In [29]:
# iframe 안에 내용을 바로 읽을 수 없음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup.select_one('body').prettify())

<body>
 <iframe allowfullscreen="true" id="mainFrame" name="mainFrame" onload="oFramesetTitleController.start(self.frames['mainFrame'], self, sTitle);oFramesetTitleController.onLoadFrame();oFramesetUrlController.start(self.frames['mainFrame']);oFramesetUrlController.onLoadFrame()" scrolling="auto" src="/PostView.naver?blogId=kidart01&amp;logNo=223095054870&amp;redirect=Dlog&amp;widgetTypeCall=true&amp;directAccess=false">
 </iframe>
</body>


In [49]:
# 프레임을 선택 (name 안쪽)
driver.switch_to.frame('mainFrame')

In [50]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
len(html)

415227

In [ ]:
# 날짜 span.se_publishDate
# 제목 div.se-title-text
# 내용 div.se-main-container
# URL

In [32]:
# URL
driver.current_url

'https://blog.naver.com/kidart01/223095054870'

In [34]:
soup.select_one('span.se_publishDate').get_text()

'2023. 5. 6. 21:00'

In [38]:
soup.select_one('div.se-title-text').get_text().strip()

'추천할만한 제주도 맛집 리스트'

In [37]:
soup.select_one('div.se-main-container').get_text()

'\n\n\n\n\n\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\u200b\u200b오랜만에 동창 모임에서 제주 여행을 떠나게 되었습니다. \u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\u200b간 김에 꼭 한번 가보고 싶던 제주도 맛집 리스트에 있는 산지물을 방문했는데요. 다양하고 다채로운 향토음식에 절로 빠져버리고 말았던 날이었어요.\u200b\u200b\n\n\n\n \n\n\n50m© NAVER Corp.더보기 /OpenStreetMap지도 데이터x© NAVER Corp. /OpenStreetMap지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가\n\n\n\n\n산지물\n제주특별자치도 제주시 임항로 26\n\n\n\n\n\n \n\n\n\n\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n산지물주소 : 제주 제주시 임항로 26영업시간 : 매일 07:30 ~ 22:30                    연중무휴전화번호 : 064-752-5599\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n주문 메뉴 : 갈치조림(소) 40,000원 고등어구이 17,000원 전복죽(중) 13,000원 자리물회 15,000원 장대 미역국 9,000원\u200b\u200b\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b이날 방문했던 음식점은 제주공항에서 차로 10분 거리에 위치한 향토음식 전문점입니다. 탑동 광장 바로 맞은편에 위치해 있어서 찾아가기도 수월했어요.\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\u200b매장 뒤편으로는 널찍한 공간의 전용 주차장이 마련되어 있어서 금세 차를 대고 안으로 들어갈 수 있었습니다.\u200b\n\n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n

In [19]:
def get_content(driver):
    driver.switch_to.frame('mainFrame')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        # 날짜
        date = soup.select_one('span.se_publishDate').get_text()
    except:
        date = ''
    try:    
        # 제목
        title = soup.select_one('div.se-title-text').get_text().strip()
    except:
        title = ''
    try:    
        # 내용
        content = soup.select_one('div.se-main-container').get_text()
        content = re.sub('[^0-9가-힣a-zA-Z.,:~#-?!]',' ',content)
        content = re.sub(' +',' ',content)
    except:
        content = ''

    # URL
    url = driver.current_url
        
    data = [date, title, content, url]

    return data

IndentationError: expected an indented block after 'except' statement on line 19 (366571858.py, line 22)

In [8]:
results = []
num = 10

for i in range(num):
    driver.get(df_url['blog_url'][i])
    time.sleep(2)

    blog_data = get_content(driver)
    results.append(blog_data)

results[9]

['2023. 5. 6. 3:15',
 '제주도 맛집 환상의 맛 오멍가멍흑돼지 & 미영이네',
 ' 드디어 그토록 바라던 제주도에 가게 되었어요. 그렇게 온지 이틀째 되던날 처음왔을땐 너무 지쳐서 밥도 맛있는거 못먹고간단하게 때우고 그랬는데 이번엔 맛있는걸 먹어야겠더라구요. 근데 아는 맛집이 없어서 어떡하지 하는데 주변에서 강추해주던 제주도 맛집 두곳을 소개해드릴까해요. 진짜 회도 싱싱하고 저녁엔 든든한 흑돼지를 먹었는데 아직도 잊을 수 가 없네요. 둘다 정말 인기있는 맛집이라서 한번 방문해보시면 후회할 일 없이 정말 좋아하실거에요. 50m NAVER Corp.더보기 /OpenStreetMap지도 데이터x NAVER Corp. /OpenStreetMap지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가 미영이네 제주특별자치도 서귀포시 대정읍 하모항구로 42 미영이네주소 : 제주 서귀포시 대정읍 하모항구로 42영업시간 : 11:30 - 22:00정기휴무 : 매주 수요일라스트오더 : 20:30전화번호 : 064-792-0077 제주도하면 회가 떠오르잖아요. 낮에는 우선 회를 먹으러 왔어요.이름이 정말 친근한 미영이네는 정말 유명하다고 해요. 모르면 간첩이라는데 여기 회가 정말 싱싱하다고 하더라구요.주차시설도 나름 괜찮아서 너무 만족스러웠던 가게에요.가면 순번으로 예약하는 기계가 있어서 그걸 먼저 뽑고 기다리면 되요. 생각보다 엄청 기다리지 않아서 좋았어요.요즘 웨이팅 하는 가게들이 굉장히 많아졌는데맛있어도 오래기다려서 먹는 걸 좋아할 만큼 인내심이 있진 않아서회전율이 빠른 게 굉장히 좋더라구요. 미영이네 메뉴고등어회대+탕 85,000고등어회소+탕 60,000방어회 변동고등어구이 15,000 미영이네는 내부가 정말 깔끔하고 인테리어도 깔끔하니 좋더라구요.저희는 방어를 먹을까 고민하다가 고등어회를 먹게 되었어요. 고등어회에 고등어구이까지 먹었는데 정말 양이 많아서 배터질거 같더라구요.진짜 회를 잘먹는 저인데도 양이 정말 많았어요.기본찬도 깔끔하게 나오면서 고등어의

In [9]:
len(results)

10

In [10]:
df = pd.DataFrame(results)
df.columns = ['날짜', '제목', '내용', 'URL']
df.to_excel('블로그내용_10개.xlsx')